# Lecture 15
- asignment 6 posted
- project 2 coming up
 



<pre>
recursion in a sane manner
y combinators is not sane

second class feature means it is not a core in the language you are working in

we want first class support for recursion

handle recursion as a first class construct

how does first class support look
    
    this says whatever we are doing is a recursive binding
    let rec f = function(x)
            if(x leq 0)
                1
            else
                x * f(x-1)
        in
            f(10)
            
rec changes the scoping rules

not to deal with circulations

val x = x - 1

scala lets us do this 

restrict ourselves to let rec should only be used to define functions 

let rec f = function(x)

rhs has to be a function definition, will enforce this


defining abstract syntax

expr => const
        ident
        plus
        geq
        .
        .
        .
        let(string, expr, expr)
        .
        .
        .
        funcdef(string, expr)
        funcall(expr, expr)
        .
        .
        .
        letRec()
        let rec nameoffunc = function(formal-parameter)
                                expr
                        in 
                            expr
                            
letrec(string,string,expr,expr)
strin: name of func
string:formal paramter
expr: first function expr
expr: in body


one example
    
    let rec f = function(x) 
                    f(x-1)
            in
                f(10)
                
                
abstract syntax
        letrec(f, x, funcall(ident(f), minus(x,1)), funcall(ident(f), const(10)))

why couldnt we do this the old way


let f = function(x)     |   let rec f = function(x)
        f(x-1)          |             f(x-1)
    in                  |          in
        f(10)           |             f(10)
        
problem is there is no               f evaluates to 
causes empty environment             closure("x", f(x-1), env)
f doesnt map to anything             needs to tell environment what f is, "special environment"
        
        
 
 
 
 
</pre>

In [2]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr

/* Here is the new stuff */
case class LetRec(funName: String, param: String, funExpr: Expr, bodyExpr: Expr) extends Expr


defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Eq
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec

In [14]:
/*-- CUT AND PASTE CODE FROM NOTEBOOK on Functions --*/
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/* -- Let us add Closure to the set of values --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


def evalExpr(e: Expr, env: Map[String, Value]): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (env contains x) 
                env(x)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
       
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
    
    
      
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
    }
}




defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined function evalExpr

In [15]:
//define own data structure for environments to allow circularity

sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/* -- Let us add Closure to the set of values --*/
case class Closure(x: String, e: Expr, pi: Environment) extends Value
case object ErrorValue extends Value

sealed trait Environment {
    def lookup(x: String): Value //defined signature
}
case class EmptyEnv() extends Environment {
    //have to define atleast one function
    //define lookup
    def lookup(x: String): Value = {
        throw new IllegalArgumentException(s"I know nothing about no string $x")
    }
}
case class Extend(x: String, v: Value, existing_env: Environment ) extends Environment {
    override def lookup(what: String ): Value = {
        if (what == x){v}
        else{
            existing_env.lookup(what)
        }
    }
}
case class ExtendRec(rec_fun_name: String, formal_param: String, body: Expr, existing_env: Environment) extends Environment {
    override def lookup(what: String): Value = {
        if( what != rec_fun_name){
            existing_env.lookup(what)
        } else {
            Closure(formal_param, body, this )
        }
    }
}

// this allows you to refer back to Extend class


defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined trait Environment
defined class EmptyEnv
defined class Extend
defined class ExtendRec

environment can be empty  
or environment can extend a existing environment by mapping x to a value  
extend("x", value, emptyenv)  
["x"->10]->["y"->0]->[]

In [13]:
val empty = EmptyEnv()
val env1 = Extend("x", NumValue(10), empty)
val env2 = Extend("y", NumValue(10), env1)

empty: EmptyEnv = EmptyEnv()
env1: Extend = Extend("x", NumValue(10.0), EmptyEnv())
env2: Extend = Extend(
  "y",
  NumValue(10.0),
  Extend("x", NumValue(10.0), EmptyEnv())
)